<a href="https://colab.research.google.com/github/sail234/ChatGLM-Tuning/blob/master/%E2%80%9CFineTurnGLM6B_ipynb%E2%80%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/mymusise/ChatGLM-Tuning.git
%cd  ChatGLM-Tuning
!pip install -r requirements.txt 

Cloning into 'ChatGLM-Tuning'...
remote: Enumerating objects: 137, done.
remote: Counting objects: 100% (67/67), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 137 (delta 41), reused 33 (delta 33), pack-reused 70
Receiving objects: 100% (137/137), 8.02 MiB | 21.28 MiB/s, done.
Resolving deltas: 100% (71/71), done.
/content/ChatGLM-Tuning
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-mxt69utf
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-mxt69utf
  Resolved https://github.com/huggingface/peft.git to commit df71b84341ae1ab3bc9b0d5f906d7a524850b63b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.3/76.3 MB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [2]:
# !python cover_alpaca2jsonl.py \
#     --data_path data/alpaca_data.json \
#     --save_path data/alpaca_data.jsonl

In [13]:
# !python tokenize_dataset_rows.py \
#     --jsonl_path data/alpaca_data.jsonl \
#     --save_path data/alpaca \
#     --max_seq_length 128
!locale-gen zh_CN.UTF-8
!update-locale LANG=zh_CN.UTF-8
!python tokenize_dataset_rows.py \
    --jsonl_path data/民法典20200528.jsonl \
    --save_path data/民法典 \
    --max_seq_length 1024

    

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:5                                                                                    │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/google/colab/_shell.py:98 in system                       │
│                                                                                                  │
│    95 │   if pip_warn:                                                                           │
│    96 │     kwargs.update({'also_return_output': True})                                          │
│    97 │                                                                                          │
│ ❱  98 │   output = _system_commands._system_compat(self, *args, **kwargs)  # pylint:disable=pr   │
│    99 │                                                                                          │
│   100 │   if pip_warn:                                                                           │
│   101 │     _pip.print_previous_import_warning(output)                                           │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/google/colab/_system_commands.py:453 in _system_compat    │
│                                                                                                  │
│   450   # We set a higher depth than the IPython system command since a shell object             │
│   451   # is expected to call this function, thus adding one level of nesting to the             │
│   452   # stack.                                                                                 │
│ ❱ 453   result = _run_command(                                                                   │
│   454 │     shell.var_expand(cmd, depth=2), clear_streamed_output=False                          │
│   455   )                                                                                        │
│   456   shell.user_ns['_exit_code'] = result.returncode                                          │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/google/colab/_system_commands.py:167 in _run_command      │
│                                                                                                  │
│   164   """Calls the shell command, forwarding input received on the stdin_socket."""            │
│   165   locale_encoding = locale.getpreferredencoding()                                          │
│   166   if locale_encoding != _ENCODING:                                                         │
│ ❱ 167 │   raise NotImplementedError(                                                             │
│   168 │   │   'A UTF-8 locale is required. Got {}'.format(locale_encoding)                       │
│   169 │   )                                                                                      │
│   170                                                                                            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968

In [4]:
import sys
sys.path.append("../")

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [24]:
from transformers import AutoTokenizer, AutoModel, TrainingArguments, AutoConfig
from modeling_chatglm import ChatGLMForConditionalGeneration
import torch
import torch.nn as nn
from peft import get_peft_model, LoraConfig, TaskType


class CastOutputToFloat(nn.Sequential):
    def forward(self, x): return super().forward(x).to(torch.float32)


model = ChatGLMForConditionalGeneration.from_pretrained("THUDM/chatglm-6b", load_in_8bit=True, trust_remote_code=True, device_map='auto')
model.supports_gradient_checkpointing = True
model.gradient_checkpointing_enable()
model.enable_input_require_grads()
model.lm_head = CastOutputToFloat(model.lm_head)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:12                                                                                   │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/transformers/modeling_utils.py:2591 in from_pretrained    │
│                                                                                                  │
│   2588 │   │   │   │   │   key: device_map[key] for key in device_map.keys() if key not in modu  │
│   2589 │   │   │   │   }                                                                         │
│   2590 │   │   │   │   if "cpu" in device_map_without_lm_head.values() or "disk" in device_map_  │
│ ❱ 2591 │   │   │   │   │   raise ValueError(                                                     │
│   2592 │   │   │   │   │   │   """                                                               │
│   2593 │   │   │   │   │   │   Some modules are dispatched on the CPU or the disk. Make sure yo  │
│   2594 │   │   │   │   │   │   the quantized model. If you want to dispatch the model on the CP  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: 
                        Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to 
fit
                        the quantized model. If you want to dispatch the model on the CPU or the disk while keeping
                        these modules in 32-bit, you need to set `load_in_8bit_fp32_cpu_offload=True` and pass a 
custom
                        `device_map` to `from_pretrained`. Check
                        https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-
cpu-and-gpu
                        for more details.

# 新段落

In [6]:
tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm-6b", trust_remote_code=True)

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


Test before Fineturn

In [7]:
# from cover_alpaca2jsonl import format_example
# import json


# instructions = json.load(open("data/alpaca_data.json"))


# with torch.no_grad():
#     for idx, item in enumerate(instructions[:5]):
#         feature = format_example(item)
#         input_text = feature["context"]
#         input_ids = tokenizer.encode(input_text, return_tensors='pt')
#         out = model.generate(
#             input_ids=input_ids,
#             max_length=150,
#             temperature=0
#         )
#         answer = tokenizer.decode(out[0])
#         print(answer)
#         item['infer_answer'] = answer
#         print(f"### {idx+1}.Answer:\n", item.get('output'), '\n\n')

In [8]:
from cover_alpaca2jsonl import format_example
import json


with torch.no_grad():
  # for idx, item in enumerate(instructions[:5]):
    # feature = format_example(item)
  input_text = "2020年颁布的民法典中关于婚姻的条文有哪些？"
  input_ids = tokenizer.encode(input_text, return_tensors='pt')
  out = model.generate(
      input_ids=input_ids,
      max_length=150,
      temperature=0
  )
  answer = tokenizer.decode(out[0])
  print(answer)

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1201: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1374: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


2020年颁布的民法典中关于婚姻的条文有哪些? 抱歉,作为一个人工智能助手,我没有访问互联网的能力,因此无法了解时事新闻,因此我无法确定2021年1月1日是否存在相关的法律法规或政策,也无法提供最新的信息。建议您咨询相关的法律专业人士或相关部门以获取最新信息。


# 新段落

In [9]:
from transformers import Trainer, HfArgumentParser

#将回答的部分Mask
def get_masks_and_position_ids(
    seq, seq_len, context_length, device, gmask=False, position_encoding_2d=True
):
    mask_position = (
        seq_len - 2
    )  # is equal to `seq.index(mask_token)` or `seq.index(150001)`
    attention_mask = torch.ones((1, context_length, context_length), device=device)
    attention_mask.tril_() #返回下三角矩阵为1，上三角为0的矩阵
    attention_mask[..., : mask_position - 1] = 1 
    attention_mask = (attention_mask < 0.5).bool()

    if position_encoding_2d:
        seq_length = seq_len - 1  # is equal to `seq_length = seq.index(150004)`
        position_ids = torch.arange(context_length, dtype=torch.long, device=device)
        if not gmask:
            position_ids[seq_length:] = mask_position
        # target的postion ids
        block_position_ids = torch.cat(
            (
                torch.zeros(seq_length, dtype=torch.long, device=device),
                torch.arange(
                    context_length - seq_length, dtype=torch.long, device=device
                )
                + 1,
            )
        )
        position_ids = torch.stack((position_ids, block_position_ids), dim=0)
    else:
        position_ids = torch.arange(context_length, dtype=torch.long, device=device)
        if not gmask:
            position_ids[context_length - 1 :] = mask_position
    return attention_mask, position_ids


def data_collator(features: list) -> dict:
    len_ids = [len(feature["input_ids"]) for feature in features] #promtion + target长度
    longest = max(len_ids) + 1 
    input_ids = []
    attention_mask_list = []
    position_ids_list = []
    labels_list = []
    for ids_l, feature in sorted(zip(len_ids, features), key=lambda x: -x[0]):
        ids = feature["input_ids"]
        seq_len = feature["seq_len"] # promtion长度
        #promtion lens ~ end 为Label的索引
        labels = (
            [-100] * (seq_len - 1)
            + ids[(seq_len - 1) :]
            + [tokenizer.eos_token_id]
            + [-100] * (longest - ids_l - 1)
        )
        ids = ids + [tokenizer.eos_token_id] * (longest - ids_l)
        _ids = torch.LongTensor(ids)
        #对输入进行位置编码
        attention_mask, position_ids = get_masks_and_position_ids(
            ids, seq_len, longest, _ids.device, gmask=False
        )
        labels_list.append(torch.LongTensor(labels))
        input_ids.append(_ids)
        attention_mask_list.append(attention_mask)
        position_ids_list.append(position_ids)
    input_ids = torch.stack(input_ids)
    labels = torch.stack(labels_list)
    attention_mask = torch.stack(attention_mask_list)
    position_ids = torch.stack(position_ids_list)
    return {
        "input_ids": input_ids,
        "labels": labels,
        "attention_mask": attention_mask,
        "position_ids": position_ids,
    }



class ModifiedTrainer(Trainer):

    def compute_loss(self, model, inputs, return_outputs=False):
        return model(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            position_ids=inputs["position_ids"],
            labels=inputs["labels"],
        ).loss

In [10]:
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, inference_mode=False,
    r=16,
    lora_alpha=32, lora_dropout=0.1,
)

model = get_peft_model(model, peft_config)
model.is_parallelizable = True
model.model_parallel = True

In [15]:
import datasets

dataset_path = "data/民法典/"

dataset = datasets.load_from_disk(dataset_path)

# train_num = 500

mini_train_dataset = datasets.Dataset.from_dict(dataset[:])

In [16]:
training_args = TrainingArguments(
    "output",
    fp16 =True,
    gradient_accumulation_steps=1,
    per_device_train_batch_size = 1,
    learning_rate = 1e-4,
    max_steps=1700,
    logging_steps=50,
    remove_unused_columns=False,
    seed=0,
    data_seed=0,
    group_by_length=False,
)


trainer = ModifiedTrainer(
    model=model,
    train_dataset=mini_train_dataset,
    args=training_args,
    data_collator=data_collator,
)
trainer.train()

import os
def save_tunable_parameters(model, path):
    saved_params = {
        k: v.to("cpu")
        for k, v in model.named_parameters()
        if v.requires_grad
    }
    torch.save(saved_params, path)


save_tunable_parameters(model, os.path.join("output", "chatglm-lora.pt"))

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
50,3.365700
100,1.722900
150,1.366200
200,1.372500
250,1.352900
300,1.419500
350,1.364000
400,1.361300
450,1.293800
500,1.428000


/usr/local/lib/python3.9/dist-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
50,3.365700
100,1.722900
150,1.366200
200,1.372500
250,1.352900
300,1.419500
350,1.364000
400,1.361300
450,1.293800
500,1.428000


/usr/local/lib/python3.9/dist-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.9/dist-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


TrainOutput(global_step=1700, training_loss=1.2808297527537627, metrics={'train_runtime': 1951.7731, 'train_samples_per_second': 0.871, 'train_steps_per_second': 0.871, 'total_flos': 3407306021634048.0, 'train_loss': 1.2808297527537627, 'epoch': 1.35})

In [17]:
with torch.no_grad():
  # for idx, item in enumerate(instructions[:5]):
    # feature = format_example(item)
  input_text = "你是谁"
  input_ids = tokenizer.encode(input_text, return_tensors='pt')
  out = model.generate(
      input_ids=input_ids,
      max_length=150,
      temperature=0.5
  )
  answer = tokenizer.decode(out[0])
  print(answer)



/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1374: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


你是谁 抱歉,我不理解您的问题。您可以提供更多背景信息或详细说明您的问题吗?


Test after Fineturn

In [ ]:
from cover_alpaca2jsonl import format_example
import json


instructions = json.load(open("data/alpaca_data.json"))


with torch.no_grad():
    for idx, item in enumerate(instructions[:5]):
        feature = format_example(item)
        input_text = feature["context"]
        input_ids = tokenizer.encode(input_text, return_tensors='pt')
        out = model.generate(
            input_ids=input_ids,
            max_length=150,
            temperature=0
        )
        answer = tokenizer.decode(out[0])
        print(answer)
        item['infer_answer'] = answer
        print(f"### {idx+1}.Answer:\n", item.get('output'), '\n\n')